# Loop 3 Analysis: External Sources & Ensemble Strategy

Key insight from jonathanchan kernel: Top solutions come from ensembling 16+ external sources.

Let's:
1. Download and score external submissions
2. Compare per-N scores with our current best
3. Build an ensemble from the best per-N configurations

In [ ]:
import pandas as pd
import numpy as np
import math
from numba import njit
import os
import subprocess

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

@njit
def score_group(xs, ys, degs, tx, ty):
    n = xs.size
    V = tx.size
    mnx = mny = 1e300
    mxx = mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c, s = math.cos(r), math.sin(r)
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xs[i]
            Y = s * tx[j] + c * ty[j] + ys[i]
            mnx, mxx = min(mnx, X), max(mxx, X)
            mny, mxy = min(mny, Y), max(mxy, Y)
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

def strip(a):
    return np.array([float(str(v).replace('s', '')) for v in a], np.float64)

def calculate_scores_by_n(df):
    scores = {}
    for n in range(1, 201):
        mask = df['id'].str.startswith(f'{n:03d}_')
        group = df[mask]
        if len(group) != n:
            continue
        xs = strip(group['x'].values)
        ys = strip(group['y'].values)
        degs = strip(group['deg'].values)
        scores[n] = score_group(xs, ys, degs, TX, TY)
    return scores

def calculate_total_score(df):
    scores = calculate_scores_by_n(df)
    return sum(scores.values()), scores

print('Scoring functions defined')

In [ ]:
# Load our current best submission
our_best_path = '/home/nonroot/snapshots/santa-2025/21198927060/code/submission.csv'
df_ours = pd.read_csv(our_best_path)
our_total, our_scores = calculate_total_score(df_ours)
print(f'Our best score: {our_total:.6f}')

In [ ]:
# Download and score external submission from GitHub
import urllib.request

external_url = 'https://raw.githubusercontent.com/SmartManoj/Santa-Scoreboard/main/submission.csv'
external_path = '/tmp/external_github.csv'

try:
    urllib.request.urlretrieve(external_url, external_path)
    df_external = pd.read_csv(external_path)
    ext_total, ext_scores = calculate_total_score(df_external)
    print(f'External (GitHub) score: {ext_total:.6f}')
    print(f'Rows: {len(df_external)}')
except Exception as e:
    print(f'Failed to download: {e}')
    ext_total, ext_scores = None, {}

In [ ]:
# Compare per-N scores
if ext_scores:
    print('\nPer-N comparison (showing where external is better):')
    print('=' * 60)
    
    better_count = 0
    total_improvement = 0
    
    for n in range(1, 201):
        if n in our_scores and n in ext_scores:
            diff = our_scores[n] - ext_scores[n]
            if diff > 1e-9:  # External is better
                better_count += 1
                total_improvement += diff
                if diff > 0.001:  # Only show significant improvements
                    print(f'N={n:3d}: ours={our_scores[n]:.6f}, ext={ext_scores[n]:.6f}, diff={diff:.6f}')
    
    print(f'\nExternal is better for {better_count}/200 N values')
    print(f'Total potential improvement: {total_improvement:.6f}')

In [ ]:
# Check more snapshots for different solutions
import glob

snapshot_dirs = sorted(glob.glob('/home/nonroot/snapshots/santa-2025/*/code/submission.csv'))
print(f'Found {len(snapshot_dirs)} snapshot submissions')

# Score a sample of snapshots to find diversity
scores_by_snapshot = {}
for path in snapshot_dirs[:20]:  # Check first 20
    try:
        df = pd.read_csv(path)
        total, _ = calculate_total_score(df)
        scores_by_snapshot[path] = total
    except:
        pass

print('\nSnapshot scores:')
for path, score in sorted(scores_by_snapshot.items(), key=lambda x: x[1])[:10]:
    print(f'  {score:.6f}: {path.split("/")[-3]}')

In [ ]:
# Build ensemble: take best per-N from all sources
print('\n' + '=' * 60)
print('BUILDING ENSEMBLE')
print('=' * 60)

# Collect all sources
sources = {
    'ours': df_ours,
    'external_github': df_external if ext_scores else None
}

# Add top snapshots
for path in list(scores_by_snapshot.keys())[:5]:
    name = path.split('/')[-3]
    try:
        sources[name] = pd.read_csv(path)
    except:
        pass

print(f'Sources: {list(sources.keys())}')

# Calculate per-N scores for all sources
all_scores = {}
for name, df in sources.items():
    if df is not None:
        _, scores = calculate_total_score(df)
        all_scores[name] = scores

print(f'Scored {len(all_scores)} sources')

In [ ]:
# Find best per-N
best_per_n = {}
best_source_per_n = {}

for n in range(1, 201):
    best_score = float('inf')
    best_source = None
    
    for name, scores in all_scores.items():
        if n in scores and scores[n] < best_score:
            best_score = scores[n]
            best_source = name
    
    if best_source:
        best_per_n[n] = best_score
        best_source_per_n[n] = best_source

ensemble_total = sum(best_per_n.values())
print(f'\nEnsemble total score: {ensemble_total:.6f}')
print(f'Our best: {our_total:.6f}')
print(f'Improvement: {our_total - ensemble_total:.6f}')

# Show source distribution
from collections import Counter
source_counts = Counter(best_source_per_n.values())
print(f'\nSource distribution:')
for source, count in source_counts.most_common():
    print(f'  {source}: {count} N values')

In [ ]:
# Build the actual ensemble submission
ensemble_rows = []

for n in range(1, 201):
    source_name = best_source_per_n.get(n)
    if source_name:
        df = sources[source_name]
        mask = df['id'].str.startswith(f'{n:03d}_')
        group = df[mask].copy()
        ensemble_rows.append(group)

df_ensemble = pd.concat(ensemble_rows, ignore_index=True)

# Sort by id
df_ensemble['sn'] = df_ensemble['id'].str.split('_').str[0].astype(int)
df_ensemble['si'] = df_ensemble['id'].str.split('_').str[1].astype(int)
df_ensemble = df_ensemble.sort_values(['sn', 'si']).drop(columns=['sn', 'si'])
df_ensemble = df_ensemble[['id', 'x', 'y', 'deg']]

print(f'Ensemble rows: {len(df_ensemble)}')
print(f'Expected: 20100')

# Verify score
ensemble_verify, _ = calculate_total_score(df_ensemble)
print(f'Verified ensemble score: {ensemble_verify:.6f}')

In [ ]:
# Save ensemble
os.makedirs('/home/code/experiments/004_ensemble_external', exist_ok=True)
ensemble_path = '/home/code/experiments/004_ensemble_external/submission.csv'
df_ensemble.to_csv(ensemble_path, index=False)
print(f'Saved to {ensemble_path}')

# Also copy to /home/submission
import shutil
os.makedirs('/home/submission', exist_ok=True)
shutil.copy(ensemble_path, '/home/submission/submission.csv')
print('Copied to /home/submission/submission.csv')